In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import json
import math

In [12]:
places_df = pd.read_json("./todo_add.json")
places_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   location_id     117 non-null    int64  
 1   place_name      117 non-null    object 
 2   city            117 non-null    object 
 3   rating          117 non-null    float64
 4   latitude        117 non-null    float64
 5   longitude       117 non-null    float64
 6   opening_hours   109 non-null    object 
 7   contact_number  49 non-null     object 
 8   photo           103 non-null    object 
 9   address         117 non-null    object 
 10  min_price       117 non-null    int64  
 11  max_price       117 non-null    int64  
 12  activity        117 non-null    object 
 13  category        117 non-null    object 
 14  description     117 non-null    object 
dtypes: float64(3), int64(3), object(9)
memory usage: 13.8+ KB


In [19]:
def get_recommendations_tf(user_activity, user_category, user_latitude, user_longitude):
    # Define the user's preferences
    user_activity = tf.constant(user_activity, dtype=tf.string)
    user_category = tf.constant(user_category, dtype=tf.string)
    user_latitude = tf.constant(user_latitude, dtype=tf.float32)
    user_longitude = tf.constant(user_longitude, dtype=tf.float32)

    # Define the places with their activity, category, latitude, and longitude
    places = places_df

    # Calculate the distance between the user's location and the places' locations
    places["distance"] = tf.sqrt(
        tf.square(places["latitude"] - user_latitude) + tf.square(places["longitude"] - user_longitude)
    )

    # Filter the places based on the user's preferences
    # filtered_places = [
    #     place for index, place in places_df.iterrows()
    #     if place["activity"] == user_activity and place["category"] == user_category
    # ]
    filtered_places = places[
        (places["activity"] == user_activity) & (places["category"] == user_category)
    ]

    # Sort the filtered places by distance
    # sorted_places = sorted(filtered_places, key=lambda x: x["distance"])
    sorted_places = filtered_places.sort_values(by="distance")

    # Find the indices of the top k recommendations
    k = 3
    # top_indices = tf.argsort(distance)[:k]
    top_indices = tf.argsort(sorted_places["distance"])[:k]

    # Gather the top k recommendations from the sorted places tensor
    # top_recommendations = tf.gather(sorted_places, top_indices)
    top_recommendations = sorted_places.iloc[top_indices]

    # Print the recommendations
    # for recommendation in top_recommendations:
    #     print(recommendation)
    for index, recommendation in top_recommendations.iterrows():
        print(recommendation)

In [16]:
# Example usage
user_activity = "Outdoor"
user_category = "Beach"
user_latitude = -1.2654
user_longitude = 116.8311

In [21]:
get_recommendations_tf(user_activity, user_category, user_latitude, user_longitude)

location_id                                                 8858076
place_name                                            Melawai Beach
city                                                     Balikpapan
rating                                                          3.5
latitude                                                   -1.27744
longitude                                                  116.8275
opening_hours     {'week_ranges': [[{'open_time': 800, 'close_ti...
contact_number                                                 None
photo             {'images': {'small': {'width': '150', 'url': '...
address            Jl. Jendral Sudirman, Balikpapan 76112 Indonesia
min_price                                                      5000
max_price                                                     15000
activity                                                    Outdoor
category                                                      Beach
description       Keindahan Pantai Melawai adala